### PACKAGES

In [ ]:
import pandas as pd
import mysql.connector as sql
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st
from streamlit_option_menu import option_menu
import requests
import json

### SQL CONNECTION

In [ ]:
mydb = sql.connect(host="localhost",
                   user="root",
                   password="#Saravanan27",
                   database= "phonepe_data"
                  )
mycursor = mydb.cursor(buffered=True)

### SQL QUERY FROM DATABASE TO DF (TABLES)

In [ ]:
# Aggregated_Insurance

mycursor.execute("select * from aggr_insurance;")
mydb.commit()
table7 = mycursor.fetchall()
Aggre_Insurance = pd.DataFrame(table7,columns = ("State", "Year", "Quarter", "Transaction_Type", "Transaction_Count", "Transaction_Amount"))

#Aggregated_transaction
mycursor.execute("select * from aggr_Transaction;")
mydb.commit()
table1 = mycursor.fetchall()
Aggre_transaction = pd.DataFrame(table1,columns = ("State", "Year", "Quarter", "Transaction_Type", "Transaction_Count", "Transaction_Amount"))

#Aggregated_user
mycursor.execute("select * from aggr_user;")
mydb.commit()
table2 = mycursor.fetchall()
Aggre_user = pd.DataFrame(table2,columns = ("States", "Year", "Quarter", "Brands", "Transaction_Count", "Percentage"))

#Map_transaction
mycursor.execute("select * from map_transaction;")
mydb.commit()
table3 = mycursor.fetchall()
Map_transaction = pd.DataFrame(table3,columns = ("State", "Year", "Quarter", "District", "Transaction_Count", "Transaction_Amount"))

#Map_Insurance
mycursor.execute("select * from map_insurance;")
mydb.commit()
table8 = mycursor.fetchall()
map_Insurance = pd.DataFrame(table8,columns = ("State", "Year", "Quarter", "District", "Transaction_Count", "Transaction_Amount"))

#Map_user
mycursor.execute("select * from map_user;")
mydb.commit()
table4 = mycursor.fetchall()
Map_user = pd.DataFrame(table4,columns = ("State", "Year", "Quarter", "District", "RegisteredUser", "AppOpens"))

#Top_transaction
mycursor.execute("select * from top_transaction;")
mydb.commit()
table5 = mycursor.fetchall()
Top_transaction = pd.DataFrame(table5,columns = ("State", "Year", "Quarter", "Pincode", "Transaction_Count", "Transaction_Amount"))

#Top_user
mycursor.execute("select * from top_user;")
mydb.commit()
table6 = mycursor.fetchall()
Top_user = pd.DataFrame(table6, columns = ("State", "Year", "Quarter", "Pincode", "RegisteredUser"))

#Top_Insurance
mycursor.execute("select * from top_insurance;")
mydb.commit()
table9 = mycursor.fetchall()
Top_Insurance = pd.DataFrame(table9,columns = ("State", "Year", "Quarter", "Pincode", "Transaction_Count", "Transaction_Amount"))

In [ ]:
Aggre_Insurance

In [ ]:
a = Aggre_Insurance[Aggre_Insurance['Year'] == 2021]
a.reset_index(drop= True,inplace= True)

b =a.groupby("State")["Transaction_Amount"]

### DATAFRAMES

In [ ]:
#Transaction_Year_Based
def Transition_Amount_Count_Year(df,year):
    trans_amt_count_year = df[df['Year'] == year]
    trans_amt_count_year.reset_index(drop= True,inplace= True)

    trans_amt_count_year_groupBy = trans_amt_count_year.groupby("State")[["Transaction_Count","Transaction_Amount"]].sum()
    trans_amt_count_year_groupBy.reset_index(inplace= True)

    fig_Ins_Amt = px.bar(trans_amt_count_year_groupBy,x="State",y="Transaction_Amount",title= f"{year} TRANSACTION AMOUNT",width=600,height=650,color_discrete_sequence=px.colors.sequential.Aggrnyl)
    fig_Ins_Amt.show()
    st.plotly_chart(fig_Ins_Amt)

    fig_Ins_Count = px.bar(trans_amt_count_year_groupBy,x="State",y="Transaction_Count",title= f"{year} TRANSACTION COUNT",width=600,height=650,color_discrete_sequence=px.colors.sequential.Bluered_r)
    fig_Ins_Count.show()
    st.plotly_chart(fig_Ins_Count)

    url="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    response = requests.get(url)
    data1 = json.loads(response.content)
    st_nms =[feature["properties"]["ST_NM"] for feature in data1["features"]]
    st_nms.sort()

    fig_india_1=px.choropleth(trans_amt_count_year_groupBy, geojson= data1, locations= "State", featureidkey= "properties.ST_NM",
                              color= "Transaction_Amount", color_continuous_scale= "Sunsetdark",
                              range_color= (trans_amt_count_year_groupBy["Transaction_Amount"].min(),trans_amt_count_year_groupBy["Transaction_Amount"].max()),
                              hover_name= "State",title = f"{year} TRANSACTION AMOUNT",
                              fitbounds= "locations",width =600, height= 600)
    fig_india_1.update_geos(visible =False)                          
    fig_india_1.show()

    fig_india_2=px.choropleth(trans_amt_count_year_groupBy, geojson= data1, locations= "State", featureidkey= "properties.ST_NM",
                              color= "Transaction_Count", color_continuous_scale= "Sunsetdark",
                              range_color= (trans_amt_count_year_groupBy["Transaction_Count"].min(),trans_amt_count_year_groupBy["Transaction_Count"].max()),
                              hover_name= "State",title = f"{year} TRANSACTION COUNT",
                              fitbounds= "locations",width =600, height= 600)
    fig_india_2.update_geos(visible =False)                          
    fig_india_2.show()

    return trans_amt_count_year

In [ ]:
#Transaction_Quarter_Based
def Transaction_Amount_Count_Quarter(df,quarter):
    trans_amt_count_Qtr = df[df['Quarter'] == quarter]
    trans_amt_count_Qtr.reset_index(drop= True,inplace= True)

    trans_amt_count_Qtr_groupBy = trans_amt_count_Qtr.groupby("State")[["Transaction_Count","Transaction_Amount"]].sum()
    trans_amt_count_Qtr_groupBy.reset_index(inplace= True)

    fig_Ins_Amt = px.bar(trans_amt_count_Qtr_groupBy,x="State",y="Transaction_Amount",title= f"{trans_amt_count_Qtr['Year'].min()} Year {quarter} QUARTER TRANSACTION AMOUNT",width=600,height=650,color_discrete_sequence=px.colors.sequential.Burg_r)
    fig_Ins_Amt.show()
    #st.plotly_chart(fig_Ins_Amt)

    fig_Ins_Count = px.bar(trans_amt_count_Qtr_groupBy,x="State",y="Transaction_Count",title= f"{trans_amt_count_Qtr['Year'].min()} Year {quarter} QUARTER TRANSACTION COUNT",width=600,height=650,color_discrete_sequence=px.colors.sequential.Cividis_r)
    fig_Ins_Count.show()
    #st.plotly_chart(fig_Ins_Count)

    url="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
    response = requests.get(url)
    data1 = json.loads(response.content)
    st_nms =[feature["properties"]["ST_NM"] for feature in data1["features"]]
    st_nms.sort()

    fig_india_1=px.choropleth(trans_amt_count_Qtr_groupBy, geojson= data1, locations= "State", featureidkey= "properties.ST_NM",
                              color= "Transaction_Amount", color_continuous_scale= "Rainbow",
                              range_color= (trans_amt_count_Qtr_groupBy["Transaction_Amount"].min(),trans_amt_count_Qtr_groupBy["Transaction_Amount"].max()),
                              hover_name= "State",title = f"{trans_amt_count_Qtr['Year'].min()} Year {quarter} QUARTER TRANSACTION AMOUNT",
                              fitbounds= "locations",width =600, height= 600)
    fig_india_1.update_geos(visible =False)                          
    fig_india_1.show()

    fig_india_2=px.choropleth(trans_amt_count_Qtr_groupBy, geojson= data1, locations= "State", featureidkey= "properties.ST_NM",
                              color= "Transaction_Count", color_continuous_scale= "Rainbow",
                              range_color= (trans_amt_count_Qtr_groupBy["Transaction_Count"].min(),trans_amt_count_Qtr_groupBy["Transaction_Count"].max()),
                              hover_name= "State",title = f"{trans_amt_count_Qtr['Year'].min()} Year {quarter} QUARTER TRANSACTION COUNT",
                              fitbounds= "locations",width =600, height= 600)
    fig_india_2.update_geos(visible =False)                          
    fig_india_2.show()

    return trans_amt_count_Qtr

In [ ]:
#Aggregated_Transaction
#based on Transaction Type
def Aggregated_Transaction_Type(df,state):
    aggr_States_List = df[df["State"] == state]
    aggr_States_List.reset_index(drop = True,inplace = True)

    Aggr_Trans_Type_GrpBy=aggr_States_List.groupby("Transaction_Type")[["Transaction_Count","Transaction_Amount"]].sum()
    Aggr_Trans_Type_GrpBy.reset_index(inplace = True)

    col1,col2= st.columns(2)
    with col1:

        fig_Aggr_Trans_States_TC=px.bar(Aggr_Trans_Type_GrpBy,x="Transaction_Count",y="Transaction_Type",orientation ="h",color_discrete_sequence=px.colors.sequential.Aggrnyl,width = 600,title= f"{state.upper()} TRANSACTION TYPES AND TRANSACTION COUNT",height= 500)
        fig_Aggr_Trans_States_TC.show()

    with col2:
        fig_Aggr_Trans_States_TA=px.bar(Aggr_Trans_Type_GrpBy,x="Transaction_Amount",y="Transaction_Type",orientation ="h",color_discrete_sequence=px.colors.sequential.Aggrnyl,width = 600,title= f"{state.upper()} TRANSACTION TYPES AND TRANSACTION COUNT",height= 500)
        fig_Aggr_Trans_States_TA.show()

### Aggregated Insurance

In [ ]:
tac_Year=Transition_Amount_Count_Year(Aggre_Insurance,2023)

In [ ]:
Transaction_Amount_Count_Quarter(tac_Year,3)

### Aggregated Transaction 

In [ ]:
tact=Transition_Amount_Count_Year(Aggre_transaction,2023)
tact

In [ ]:
tact=Aggre_transaction[Aggre_transaction['State']=="West Bengal"]

In [ ]:
#Aggregated Transaction based on transaction Type
tacg = tact.groupby("Transaction_Type")[["Transaction_Count","Transaction_Amount"]].sum()
tacg.reset_index(inplace= True)
tacg

In [ ]:
#Aggregated Transaction Pie Chart for transactiontype DF
fig_Aggr_Trans_States_TC = px.pie(data_frame=tacg,names="Transaction_Type",values="Transaction_Amount",color_discrete_sequence=px.colors.sequential.Rainbow,hole=0.25,width = 600,title="TRANSACTION AMOUNT")
fig_Aggr_Trans_States_TC.show()

fig_Aggr_Trans_States_TC = px.pie(data_frame=tacg,names="Transaction_Type",values="Transaction_Count",color_discrete_sequence=px.colors.sequential.Rainbow,hole=0.25,width = 600,title="TRANSACTION COUNT")
fig_Aggr_Trans_States_TC.show()

### Aggregated_Transaction DF

In [ ]:
Aggre_transaction

In [ ]:
a = Aggre_transaction[Aggre_transaction["State"] == "Tamil Nadu"]
a

In [ ]:
#Aggregated_Transaction GroupBy
Aggr_Trans_Type_GrpBy=a.groupby("Transaction_Type")[["Transaction_Count","Transaction_Amount"]].sum()
Aggr_Trans_Type_GrpBy

#Aggregated User

### Aggregated User for Years

In [ ]:
aggr_user_year=Aggre_user[Aggre_user['Year'] == 2022]
aggr_user_year.reset_index(drop=True)

In [ ]:
aggr_user_year=Aggre_user[Aggre_user['Year'] == 2022]
aggr_user_year.reset_index(drop=True,inplace=True)

In [ ]:
aggr_user_year

In [ ]:
aggr_user_year_brand_grpBY=aggr_user_year.groupby("Brands")["Transaction_Count"].sum()
aggr_user_year_brand_grpBY.reset_index(drop= True)

In [ ]:
aggr_user_year_brand_grpBY=aggr_user_year.groupby("Brands")["Transaction_Count"].sum()
aggr_user_year_brand_grpBY.reset_index(drop= True,inplace= True)

In [ ]:
aggr_user_year_brand_grpBY

In [ ]:
aggr_user_year_brand_grpBY=pd.DataFrame(aggr_user_year.groupby("Brands")["Transaction_Count"].sum())
aggr_user_year_brand_grpBY.reset_index(inplace = True)

In [ ]:
aggr_user_year_brand_grpBY

In [ ]:
aggr_user_year_brand_grpBY.get("Transaction_Count")

In [ ]:
aggr_user_year_brand_grpBY

In [ ]:
aggr_user_year_brand_grpBY.reset_index(drop=True,inplace=True)

In [ ]:
fig_aggr_user_brands=px.bar(aggr_user_year_brand_grpBY, x="Brands", y= "Transaction_Count", title="BRANDS AND TRANSACTION COUNT",
                            width=800,color_discrete_sequence=px.colors.sequential.haline_r)
fig_aggr_user_brands.show()                           

In [ ]:
def Aggr_User_Year_Plot(df,year):
    aggr_user_year=df[df['Year'] == year]
    aggr_user_year.reset_index(drop=True,inplace=True)       

    aggr_user_year_brand_grpBY=pd.DataFrame(aggr_user_year.groupby("Brands")["Transaction_Count"].sum())
    aggr_user_year_brand_grpBY.reset_index(inplace = True)  

    fig_aggr_user_brands=px.bar(aggr_user_year_brand_grpBY, x="Brands", y= "Transaction_Count", title=f"{year} BRANDS AND TRANSACTION COUNT",
                            width=800,color_discrete_sequence=px.colors.sequential.haline_r)
    fig_aggr_user_brands.show()

In [ ]:
Aggr_User_Year_Plot(Aggre_user,2022)

In [ ]:
Aggre_user

In [ ]:
aggr_User_Yr_List=Aggre_user['Year'].unique()
aggr_User_Yr_List

### Aggregated User for Quarter

In [ ]:
Aggre_user[Aggre_user['Quarter'] == 1]

In [ ]:
aggr_user_year=Aggre_user[Aggre_user['Quarter'] == 1]
aggr_user_year.reset_index(drop=True)

In [ ]:
fig_pie_1= px.pie(data_frame=aggr_user_year, names= "Brands", values="Transaction_Count", hover_data= "Percentage",
                  width=1000,title="QUARTER TRANSACTION COUNT PERCENTAGE",hole=0.5, color_discrete_sequence= px.colors.sequential.Magenta_r)
fig_pie_1.show()

In [ ]:
aggr_user_year[aggr_user_year['Quarter'] == 1]

In [ ]:
aggr_user_year_Quarters=aggr_user_year[aggr_user_year['Quarter'] == 1]
aggr_user_year_Quarters

In [ ]:
aggr_user_year_Quarters=aggr_user_year[aggr_user_year['Quarter'] == 1]
aggr_user_year_Quarters.reset_index(drop=True)

In [ ]:
aggr_user_year_Quarters=aggr_user_year[aggr_user_year['Quarter'] == 1]
aggr_user_year_Quarters.reset_index(drop=True,inplace = True)

In [ ]:
fig_pie_1= px.bar(data_frame=aggr_user_year_Quarters, x= "Brands", y="Transaction_Count", hover_data= "Percentage",
                      width=1000,title=" QUARTER TRANSACTION COUNT PERCENTAGE", color_discrete_sequence= px.colors.sequential.Magenta_r)
fig_pie_1.show()                      

### Aggregated User for States

In [ ]:
Aggre_user[Aggre_user['States'] == "West Bengal"]

In [ ]:
aggr_user_States=Aggre_user[Aggre_user['States'] == "West Bengal"]
aggr_user_States

In [ ]:
aggr_user_States=Aggre_user[Aggre_user['States'] == "West Bengal"]
aggr_user_States.reset_index(drop=True)

In [ ]:
aggr_user_States=Aggre_user[Aggre_user['States'] == "West Bengal"]
aggr_user_States.reset_index(drop=True,inplace=True)

In [ ]:
aggr_user_States.groupby("Brands")["Transaction_Count"].sum()

In [ ]:
aggr_User_States_GrpBy=pd.DataFrame(aggr_user_States.groupby("Brands")["Transaction_Count"].sum())
aggr_User_States_GrpBy

In [ ]:
aggr_User_States_GrpBy=pd.DataFrame(aggr_user_States.groupby("Brands")["Transaction_Count"].sum())
aggr_User_States_GrpBy.reset_index(inplace=True)

In [ ]:
aggr_User_States_GrpBy

In [ ]:
fig_scatter_1= px.line(aggr_User_States_GrpBy, x= "Brands", y= "Transaction_Count", markers= True, width=1000)
fig_scatter_1.show()

In [ ]:
Aggre_user

### MAP INSURANCE

In [ ]:
map_Ins_State_List=map_Insurance[map_Insurance['State'] == "Tamil Nadu"]
map_Ins_State_List

In [ ]:
def map_insurance_plot_state(df,state):
    map_Ins_State_List=df[df['State'] == state]
    map_Ins_State_List.reset_index(drop=True,inplace =True)

    map_Ins_State_List_gpby=map_Ins_State_List.groupby("District")[["Transaction_Count","Transaction_Amount"]].sum()
    map_Ins_State_List_gpby.reset_index(inplace=True)

    col1,col2=st.columns(2)
    with col1:
        
        fig_map_ins_state_bar=px.bar(map_Ins_State_List_gpby, x= "Transaction_Amount", y="District",orientation = "h",height = 550,
                                     width=600,title=f"{state.upper()} DISTRICTS AND TRANSACTION AMOUNT", color_discrete_sequence= px.colors.sequential.Mint_r)
        #st.plotly_chart(fig_map_ins_state_bar)
        fig_map_ins_state_bar.show()
    with col2:

        fig_map_ins_state_bar=px.bar(data_frame=map_Ins_State_List_gpby, x= "Transaction_Count", y="District",orientation = "h",height = 550,
                                     width=550,title=f"{state.upper()} DISTRICTS AND TRANSACTION COUNT", color_discrete_sequence= px.colors.sequential.Aggrnyl_r)
        #st.plotly_chart(fig_map_ins_state_bar)  
        fig_map_ins_state_bar.show()

In [ ]:
df_map_insur_Y_Q=Transition_Amount_Count_Year(map_Insurance,2020)

In [ ]:
map_insurance_plot_state(df_map_insur_Y_Q,"Tamil Nadu")

In [ ]:
Transaction_Amount_Count_Quarter(df_map_insur_Y_Q,2)

In [ ]:
map_Insurance

### Map Transaction

In [ ]:
df_map_Trans_Y=Transition_Amount_Count_Year(Map_transaction,2023)

In [ ]:
map_insurance_plot_state(df_map_Trans_Y,"West Bengal")

In [ ]:
map_Trans_QTR_df=Transaction_Amount_Count_Quarter(df_map_Trans_Y,3)

In [ ]:
map_insurance_plot_state(map_Trans_QTR_df,"Tamil Nadu")

In [ ]:
Map_transaction

### Map USER

In [ ]:
def map_User_Year(df,year):
    map_user_yr=df[df["Year"] == year]
    map_user_yr.reset_index(drop=True,inplace=True)

    map_user_yr_gb=map_user_yr.groupby("State")[["RegisteredUser","AppOpens"]].sum()
    map_user_yr_gb.reset_index(inplace=True)

    fig_map_user_plot_1= px.line(map_user_yr_gb, x= "State", y= ["RegisteredUser","AppOpens"], markers= True,orientation="v",labels={'State': "State"},
                                 width=1000,height=600,title=f" {year} REGISTERED USERS AND APP OPENS", color_discrete_sequence= px.colors.sequential.Viridis_r)

    fig_map_user_plot_1.show()
    return map_user_yr

In [ ]:
muy=map_User_Year(Map_user,2023)

In [ ]:
def map_User_Quarter(df,quarter):
    map_user_qtr = df[df["Quarter"] == quarter]
    map_user_qtr.reset_index(drop=True,inplace=True)
    map_user_qtr_gb=map_user_qtr.groupby("State")[["RegisteredUser","AppOpens"]].sum()
    map_user_qtr_gb.reset_index(inplace=True)

    fig_map_user_plot_1= px.line(map_user_qtr_gb, x= "State", y= ["RegisteredUser","AppOpens"], markers= True,
                                    title= f"{df['Year'].min()}, {quarter} QUARTER REGISTERED USER AND APPOPENS",
                                    width= 1000,height=800,color_discrete_sequence= px.colors.sequential.Rainbow_r)

    fig_map_user_plot_1.show()

    return map_user_qtr

In [ ]:
muq=map_User_Quarter(muy,2)

In [ ]:
def map_User_State(df,state):
    map_user_state = df[df["State"] == state]
    map_user_state.reset_index(drop=True,inplace=True)
    map_user_state_gb=map_user_state.groupby("District")[["RegisteredUser","AppOpens"]].sum()
    map_user_state_gb.reset_index(inplace=True)

    col1,col2 = st.columns(2)
    with col1:
        fig_map_user_state1=px.bar(map_user_state_gb,x="RegisteredUser",y="District",orientation="h",
            title="REGISTERED USER",height=800,
            color_discrete_sequence=px.colors.sequential.Plasma_r)
        fig_map_user_state1.show()
        
    with col2:
        fig_map_user_state2=px.bar(map_user_state_gb,x="AppOpens",y="District",orientation="h",
                                title="APP OPENS",height=800,
                                color_discrete_sequence=px.colors.sequential.Oranges_r)  
        fig_map_user_state2.show()  

In [ ]:
map_User_State(muq,"Tamil Nadu")

In [ ]:
Map_user

### Top INsurance

In [ ]:
Top_Insurance

In [ ]:
top_Ins_yr = Top_Insurance[Top_Insurance['State'] == "West Bengal"]

In [ ]:
Transition_Amount_Count_Year(top_Ins_yr,2020)

In [ ]:
Top_Insurance

### TOP USER

In [ ]:
Top_transaction

In [ ]:
tt=Top_transaction[Top_transaction['State'] == "Andaman & Nicobar"]
tt.reset_index(drop=True,inplace=True)
tt.groupby(['Year', 'Quarter'])[["Transaction_Count","Transaction_Amount"]].sum()

In [ ]:
def top_user_yr(df,year):
    top_user_year=df[df['Year'] == year]
    top_user_year.reset_index(drop= True, inplace= True)
    top_user_year_gby=pd.DataFrame(top_user_year.groupby(["State","Quarter"])['RegisteredUser'].sum())
    top_user_year_gby.reset_index(inplace= True)
    fig_top_user_yr_plot= px.bar(top_user_year_gby, x= "State", y= "RegisteredUser", barmode= "group", color= "Quarter",
                                width=1000, height= 800, color_continuous_scale= px.colors.sequential.Burgyl)
    fig_top_user_yr_plot.show()
    #st.plotly_chart(fig_top_user_y

    return top_user_year

In [ ]:
def top_user_state(df,state):
    top_user_state=df[df['State'] == state ]
    top_user_state.reset_index(drop= True, inplace= True)
    top_user_state_gby=pd.DataFrame(top_user_state.groupby("Quarter")['RegisteredUser'].sum())
    top_user_state_gby.reset_index(inplace= True)
    fig_top_user_state_plot= px.bar(top_user_state_gby, x= "Quarter", y= "RegisteredUser", barmode= "group",
                                width=1000, height= 800, color_continuous_scale= px.colors.sequential.Magenta)
    fig_top_user_state_plot.show()
    #st.plotly_chart(fig_top_user_state_plot)

In [ ]:
df_top_user_Y=top_user_yr(Top_user,2020)

In [ ]:
df_top_user_Y_S=top_user_state(df_top_user_Y,"Tamil Nadu")

In [ ]:
Top_user

In [ ]:
tta=Top_user[Top_user['Year'] == 2018]
tta.reset_index(drop=True,inplace=True)

In [ ]:
state = "Tamil Nadu"
filtered_data = tta[tta["State"] == state]
result=filtered_data.groupby(["Quarter","State"])[["RegisteredUser"]].sum()
result

In [ ]:
tt=Top_user[Top_user['State'] == "Tamil Nadu"]
tt.reset_index(drop=True,inplace=True)
tt.groupby("Quarter")[["RegisteredUser"]].sum()

### QUestions

In [ ]:
Aggre_user

In [ ]:
brand= Aggre_user[["Brands","Transaction_Count"]]
brand1= brand.groupby("Brands")["Transaction_Count"].sum().sort_values(ascending=False)
brand2= pd.DataFrame(brand1).reset_index()
brand2

In [205]:
def ques1():
    brand= Aggre_user[["Brands","Transaction_Count"]]
    brand1= brand.groupby("Brands")["Transaction_Count"].sum().sort_values(ascending=False)
    brand2= pd.DataFrame(brand1).reset_index()

    fig_brands= px.pie(brand2, values= "Transaction_Count", names= "Brands", color_discrete_sequence=px.colors.sequential.dense_r,
                        title= "Top Mobile Brands of Transaction_count")
    fig_brands.show()

    return st.plotly_chart(fig_brands)

In [ ]:
ques1()

In [ ]:
Aggre_transaction

In [203]:
def ques2():
    lt= Aggre_transaction[["State", "Transaction_Amount"]]
    lt1= lt.groupby("State")["Transaction_Amount"].sum().sort_values(ascending= True)
    lt2= pd.DataFrame(lt1).reset_index().head(10)

    fig_lts= px.bar(lt2, x= "State", y= "Transaction_Amount",title= "Top 10 Lowest Transaction Amount and States",
                    color_discrete_sequence= px.colors.sequential.Oranges_r)
    fig_lts.show()
    return st.plotly_chart(fig_lts)

In [ ]:
ques2()

In [202]:
def ques3():
    htd= Map_transaction[["District", "Transaction_Amount","State"]]
    htd1= htd.groupby(["District","State"])["Transaction_Amount"].sum().sort_values(ascending=False)
    htd2= pd.DataFrame(htd1).head(10).reset_index()

    fig_htd= px.pie(htd2, values= "Transaction_Amount", names= "District", title="Top 10 Districts Of Highest Transaction Amount",
                    color_discrete_sequence=px.colors.sequential.Emrld_r,hover_name="State")
    fig_htd.show()
    return st.plotly_chart(fig_htd)

In [ ]:
ques3()

In [ ]:
Map_transaction

In [200]:
def ques4():
    htd= Map_transaction[["District", "Transaction_Amount","State"]]
    htd1= htd.groupby(["District","State"])["Transaction_Amount"].sum().sort_values(ascending=True)
    htd2= pd.DataFrame(htd1).head(10).reset_index()

    fig_htd= px.pie(htd2, values= "Transaction_Amount", names= "District", title="Top 10 Districts of Lowest Transaction Amount",
                    color_discrete_sequence=px.colors.sequential.Greens_r,hover_name="State")
    fig_htd.show()
    return st.plotly_chart(fig_htd)

In [ ]:
ques4()

In [175]:
def ques5():
    sa= Map_user[["State", "AppOpens"]]
    sa1= sa.groupby("State")["AppOpens"].sum().sort_values(ascending=False)
    sa2= pd.DataFrame(sa1).reset_index().head(10)

    fig_sa= px.bar(sa2, x= "State", y= "AppOpens", title="Top 10 Highest States With AppOpens",
                color_discrete_sequence= px.colors.sequential.deep_r)
    fig_sa.show()
    return st.plotly_chart(fig_sa)

In [ ]:
ques5()

In [ ]:
Map_user

In [177]:
def ques6():
    sa= Map_user[["State", "AppOpens"]]
    sa1= sa.groupby("State")["AppOpens"].sum().sort_values(ascending=True)
    sa2= pd.DataFrame(sa1).reset_index().head(10)

    fig_sa= px.bar(sa2, x= "State", y= "AppOpens", title="Top 10 Lowest States With AppOpens",
                color_discrete_sequence= px.colors.sequential.dense_r)
    fig_sa.show()
    return st.plotly_chart(fig_sa)

In [ ]:
ques6()

In [198]:
def ques7():
    stc= Aggre_transaction[["State", "Transaction_Count"]]
    stc1= stc.groupby("State")["Transaction_Count"].sum().sort_values(ascending=True)
    stc2= pd.DataFrame(stc1).head(10).reset_index()

    fig_stc= px.bar(stc2, x= "State", y= "Transaction_Count",title= "Top 10 States with Lowest Transaction Count",
                    color_discrete_sequence= px.colors.sequential.Jet_r)
    fig_stc.show()
    return st.plotly_chart(fig_stc)

In [ ]:
ques7()

In [208]:
def ques8():
    stc= Aggre_transaction[["State", "Transaction_Count"]]
    stc1= stc.groupby("State")["Transaction_Count"].sum().sort_values(ascending=False)
    stc2= pd.DataFrame(stc1).reset_index()

    fig_stc= px.bar(stc2, x= "State", y= "Transaction_Count", title= "STATES WITH HIGHEST TRANSACTION COUNT",
                    color_discrete_sequence= px.colors.sequential.Magenta_r)
    fig_stc.show()
    return st.plotly_chart(fig_stc)

In [ ]:
ques8()

In [211]:
def ques9():
    ht= Aggre_transaction[["State", "Transaction_Amount"]]
    ht1= ht.groupby("State")["Transaction_Amount"].sum().sort_values(ascending= False)
    ht2= pd.DataFrame(ht1).reset_index().head(10)

    fig_lts= px.bar(ht2, x= "State", y= "Transaction_Amount",title= "Highest 10 States With Transaction Amount",
                    color_discrete_sequence= px.colors.sequential.Oranges_r)
    fig_lts.show()
    return st.plotly_chart(fig_lts)

In [ ]:
ques9()

In [ ]:
Aggre_transaction

In [218]:
def ques10():
    dt= Map_transaction[["District", "Transaction_Amount","State"]]
    dt1= dt.groupby(["District","State"])["Transaction_Amount"].sum().sort_values(ascending=True)
    dt2= pd.DataFrame(dt1).reset_index().head(50)

    fig_dt= px.bar(dt2, x= "District", y= "Transaction_Amount", title= "Top 50 Districts With Lowest Transaction Amount",
                color_discrete_sequence= px.colors.sequential.Mint_r,hover_name="State")
    fig_dt.show()
    return st.plotly_chart(fig_dt)

In [ ]:
ques10()

In [ ]:
Map_transaction